In [1]:
import pickle
import os
from perses.app.relative_point_mutation_setup import PointMutationExecutor
import argparse
from simtk import openmm, unit
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


In [2]:
from __future__ import absolute_import

from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS', 'HID', 'HIE', 'HIP']

# Set up logger
import logging
_logger = logging.getLogger("setup")
_logger.setLevel(logging.INFO)
class PointMutationExecutor2(PointMutationExecutor):
    """
    Simple, stripped-down class to create a protein-ligand system and allow a mutation of a protein.
    this will allow support for the creation of _two_ relative free energy calculations:
        1. 'wildtype' - 'point mutant' complex hybrid.
        2. 'wildtype' - 'point mutant' protein hybrid (i.e. with ligand of interest unbound)
    Example (create full point mutation executor and run parallel tempering on both complex and apo phases):
        from pkg_resources import resource_filename
        protein_path = 'data/perses_jacs_systems/thrombin/Thrombin_protein.pdb'
        ligands_path = 'data/perses_jacs_systems/thrombin/Thrombin_ligands.sdf'
        protein_filename = resource_filename('openmmforcefields', protein_path)
        ligand_input = resource_filename('openmmforcefields', ligands_path)
        pm_delivery = PointMutationExecutor(protein_filename=protein_filename,
                                    mutation_chain_id='2',
                                    mutation_residue_id='198',
                                     proposed_residue='THR',
                                     phase='complex',
                                     conduct_endstate_validation=False,
                                     ligand_input=ligand_input,
                                     ligand_index=0,
                                     forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                                     barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                                     forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.PME, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                                     small_molecule_forcefields='gaff-2.11')
        complex_htf = pm_delivery.get_complex_htf()
        apo_htf = pm_delivery.get_apo_htf()
        # Now we can build the hybrid repex samplers
        from perses.annihilation.lambda_protocol import LambdaProtocol
        from openmmtools.multistate import MultiStateReporter
        from perses.samplers.multistate import HybridRepexSampler
        from openmmtools import mcmc
        suffix = 'run'; selection = 'not water'; checkpoint_interval = 10; n_states = 11; n_cycles = 5000
        for htf in [complex_htf, apo_htf]:
            lambda_protocol = LambdaProtocol(functions='default')
            reporter_file = 'reporter.nc'
            reporter = MultiStateReporter(reporter_file, analysis_particle_indices = htf.hybrid_topology.select(selection), checkpoint_interval = checkpoint_interval)
            hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                  collision_rate=5.0 / unit.picosecond,
                                                                                  n_steps=250,
                                                                                  reassign_velocities=False,
                                                                                  n_restart_attempts=20,
                                                                                  splitting="V R R R O R R R V",
                                                                                  constraint_tolerance=1e-06),
                                                                                  hybrid_factory=htf, online_analysis_interval=10)
            hss.setup(n_states=n_states, temperature=300*unit.kelvin, storage_file=reporter, lambda_protocol=lambda_protocol, endstates=False)
            hss.extend(n_cycles)
    """
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 old_residue=None,
                 phase='complex',
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 allow_undefined_stereo_sdf=False,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 generate_unmodified_hybrid_topology_factory=True,
                 generate_rest_capable_hybrid_topology_factory=False,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            old_residue : str, default None
                Use this argument to specify the name of the old residue if it should correspond to a nonstandard amino acid (e.g. LYN, ASH, HID, etc)
                If not specified, the old residue name is inferred to be the name in the input PDB.
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_input : str, default None
                path to ligand of interest (i.e. small molecule or protein); .sdf or .pdb
            ligand_index : int, default 0
                which ligand to use
            allow_undefined_stereo_sdf : bool, default False
                whether to allow an SDF file to contain undefined stereocenters
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            generate_unmodified_hybrid_topology_factory : bool, default True
                whether to generate a vanilla HybridTopologyFactory
            generate_rest_capable_hybrid_topology_factory : bool, default False
                whether to generate a RepartitionedHybridTopologyFactory
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """
        from openeye import oechem
        
        # Add nonstandard amino acid bond definitions
#         app.Topology.loadBondDefinitions("residues_nonstandard.xml")
        

        # First thing to do is load the apo protein to mutate...
        protein_pdbfile = open(protein_filename, 'r')
        protein_pdb = app.PDBFile(protein_pdbfile)
        protein_pdbfile.close()
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_topology = protein_md_topology.to_openmm()
        protein_n_atoms = protein_md_topology.n_atoms

        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_file, index=ligand_index, allow_undefined_stereo=allow_undefined_stereo_sdf)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                if ligand_input.endswith('pdb'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            # Now create a complex
            complex_md_topology = protein_md_topology.join(ligand_md_topology)
            complex_topology = complex_md_topology.to_openmm()
            complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
            complex_positions[:protein_n_atoms, :] = protein_positions
            complex_positions[protein_n_atoms:, :] = ligand_positions

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex...
        apo_input = list(self._solvate(protein_topology, protein_positions, water_model, phase, ionic_strength, apo_box_dimensions))
        inputs = [apo_input]
        if ligand_input:
            inputs.append(self._solvate(complex_topology, complex_positions, water_model, phase, ionic_strength, complex_box_dimensions))

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for is_complex, (top, pos, sys) in enumerate(inputs):
            # Change the name of the old residue to its nonstandard name, if necessary
            # Note this needs to happen after generating the system, as that uses standard residue names
            if old_residue:
                for residue in top.residues():
                    if residue.id == mutation_residue_id:
                        residue.name = old_residue
                        print(f"Changed resid {mutation_residue_id} to {residue.name}")
                
            point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                                 system_generator=self.system_generator,
                                                                 chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                 max_point_mutants=1,
                                                                 residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                 allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                 aggregate=True) # Always allow aggregation

            topology_proposal = point_mutation_engine.propose(sys, top)
#             self.topology_proposal = topology_proposal
            # Only validate energy bookkeeping if the WT and proposed residues do not involve rings
            old_res = [res for res in top.residues() if res.id == mutation_residue_id][0]
            validate_bool = False if old_res.name in ring_amino_acids or proposed_residue in ring_amino_acids else True
            new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                                   validate_energy_bookkeeping=validate_bool)

            #check for charge change...
            charge_diff = point_mutation_engine._get_charge_difference(current_resname=topology_proposal.old_topology.residue_topology.name,
                                                                       new_resname=topology_proposal.new_topology.residue_topology.name)
            _logger.info(f"charge diff: {charge_diff}")
            if charge_diff != 0:
                new_water_indices_to_ionize = point_mutation_engine.get_water_indices(charge_diff, new_positions, topology_proposal.new_topology, radius=0.8)
                _logger.info(f"new water indices to ionize {new_water_indices_to_ionize}")
                self._get_ion_and_water_parameters(topology_proposal.old_system, topology_proposal.old_topology)
                self._transform_waters_into_ions(new_water_indices_to_ionize, topology_proposal.new_system, charge_diff)
                PointMutationExecutor._modify_atom_classes(new_water_indices_to_ionize, topology_proposal)


            logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
                                                        validate_energy_bookkeeping=validate_bool)
            if generate_unmodified_hybrid_topology_factory:
                repartitioned_endstate = None
                self.generate_htf(HybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex)
            if generate_rest_capable_hybrid_topology_factory:
                 for repartitioned_endstate in [0, 1]:
                    self.generate_htf(RepartitionedHybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex)

            if not topology_proposal.unique_new_atoms:
                assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
                assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            else:
                added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

            if not topology_proposal.unique_old_atoms:
                assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
                assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
                subtracted_valence_energy = 0.0
            else:
                subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


            if conduct_endstate_validation and repartitioned_endstate is None:
                zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta=beta, ENERGY_THRESHOLD=ENERGY_THRESHOLD)
                if zero_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 0 state is above the threshold ({ENERGY_THRESHOLD}): {zero_state_error}")
                if one_state_error > ENERGY_THRESHOLD:
                    _logger.warning(f"Reduced potential difference of the nonalchemical and alchemical Lambda = 1 state is above the threshold ({ENERGY_THRESHOLD}): {one_state_error}")
            else:
                pass


# LYN27A

In [10]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barnase_renumbered_lyn27.pdb",
                        '1', # First chain is the barstar one
                        '27',
                        'ALA',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=CE, atomic number=6), Atom(name=NZ, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HD3, atomic number=1), Atom(name=HE2, atomic number=1), Atom(name=HE3, atomic number=1), Atom(name=HZ1, atomic number=1), Atom(name=HZ2, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposa

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 1 new atoms
INFO:geometry:	reduced angle potential = 0.026255259241927658.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 885 bond forces in the no-nonbonded final system
INFO:geometry:	there are 3078 angle forces in the no-nonbonded final system
INFO:geometry:	there are 5813 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 5680.188157741516


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -15.749000863705614
INFO:geometry:final reduced energy 5664.439155342768
INFO:geometry:sum of energies: 5664.43915687781
INFO:geometry:magnitude of difference in the energies: 1.5350428022031792e-06
INFO:geometry:Final logp_proposal: -3825.763227693969


added energy components: [('CustomBondForce', 0.41235653061941924), ('CustomAngleForce', 3.519470373761121), ('CustomTorsionForce', 4.640212543635615), ('CustomBondForce', -24.321040311721767)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [11]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/0/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [12]:
import mdtraj as md

In [13]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [14]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# ASH39A

In [3]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barstar_renumbered_ash39.pdb",
                        '1', # First chain is the barstar one
                        '39',
                        'ALA',
                        ligand_input="../../input/1brs_barnase_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=OD1, atomic number=8), Atom(name=OD2, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD2, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 1
INFO:geometry:Atom index proposal order is [637]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_posit

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 1 new atoms
INFO:geometry:	reduced angle potential = 0.29414498021917546.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 732 bond forces in the no-nonbonded final system
INFO:geometry:	there are 2632 angle forces in the no-nonbonded final system
INFO:geometry:	there are 4962 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 5670.107693081365


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 8.817409529952398
INFO:geometry:final reduced energy 5678.92510194569
INFO:geometry:sum of energies: 5678.925102611318
INFO:geometry:magnitude of difference in the energies: 6.656278639383117e-07
INFO:geometry:Final logp_proposal: -6113.552253403806
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...


added energy components: [('CustomBondForce', 10.788412589027248), ('CustomAngleForce', 1.9836023160449177), ('CustomTorsionForce', 33.1093980157991), ('CustomBondForce', -37.06400339091888)]


INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=7.062600000000001, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=7.062600000000001), unit=nanometer)]
INFO:relati

In [5]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/1/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [6]:
import mdtraj as md

In [8]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [9]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# T42A

In [18]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barstar_renumbered.pdb",
                        '1', # First chain is the barstar one
                        '42',
                        'ALA',
                        ligand_input="../../input/1brs_barnase_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 2
INFO:geometry:Atom index proposal order is [678, 677]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of propo

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 2 new atoms
INFO:geometry:	reduced angle potential = 0.7149050137355307.
INFO:geometry:	reduced angle potential = 0.039718813146820714.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 733 bond forces in the no-nonbonded final system
INFO:geometry:	there are 2628 angle forces in the no-nonbonded final system
INFO:geometry:	there are 4943 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 5676.816570044722


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -61.388432254921106
INFO:geometry:final reduced energy 5615.428137981567
INFO:geometry:sum of energies: 5615.4281377898005
INFO:geometry:magnitude of difference in the energies: 1.917660554795475e-07
INFO:geometry:Final logp_proposal: -7322.114562441837


added energy components: [('CustomBondForce', 0.334075218387163), ('CustomAngleForce', 1.7009210376719293), ('CustomTorsionForce', 10.24279269210873), ('CustomBondForce', -73.66622120308894)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [19]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/2/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [20]:
import mdtraj as md

In [21]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [22]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# Y29A

In [23]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barstar_renumbered.pdb",
                        '1', # First chain is the barstar one
                        '29',
                        'ALA',
                        ligand_input="../../input/1brs_barnase_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD1, atomic number=6), Atom(name=CD2, atomic number=6), Atom(name=CE1, atomic number=6), Atom(name=CE2, atomic number=6), Atom(name=CZ, atomic number=6), Atom(name=OH, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD1, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HE1, atomic number=1), Atom(name=HE2, atomic number=1), Atom(name=HH, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposa

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 1 new atoms
INFO:geometry:	reduced angle potential = 0.13319690860803637.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 727 bond forces in the no-nonbonded final system
INFO:geometry:	there are 2616 angle forces in the no-nonbonded final system
INFO:geometry:	there are 4929 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 5681.99072406439


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -2.9502471366034837
INFO:geometry:final reduced energy 5677.555537445395
INFO:geometry:sum of energies: 5679.040476927787
INFO:geometry:magnitude of difference in the energies: 1.4849394823910376
INFO:geometry:Final logp_proposal: 75.5735120897571


added energy components: [('CustomBondForce', 0.7568798488023687), ('CustomAngleForce', 2.4462341601754227), ('CustomTorsionForce', 3.1258130423487493), ('CustomBondForce', -9.279174187930023)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [24]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/3/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [25]:
import mdtraj as md

In [26]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [27]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# K27A

In [3]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor("../../input/1brs_barnase_renumbered.pdb",
                        '1', # First chain is the barstar one
                        '27',
                        'ALA',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=CE, atomic number=6), Atom(name=NZ, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HD3, atomic number=1), Atom(name=HE2, atomic number=1), Atom(name=HE3, atomic number=1), Atom(name=HZ1, atomic number=1), Atom(name=HZ2, atomic number=1), Atom(name=HZ3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:ge

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 1 new atoms
INFO:geometry:	reduced angle potential = 0.015531115155059803.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 885 bond forces in the no-nonbonded final system
INFO:geometry:	there are 3078 angle forces in the no-nonbonded final system
INFO:geometry:	there are 5813 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 5680.188157741516


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 43.326903873328305
INFO:geometry:final reduced energy 5723.515059095781
INFO:geometry:sum of energies: 5723.515061614844
INFO:geometry:magnitude of difference in the energies: 2.5190633863303447e-06
INFO:geometry:Final logp_proposal: -11358.231496615645
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...


added energy components: [('CustomBondForce', 0.41235653061941924), ('CustomAngleForce', 3.5195542213731335), ('CustomTorsionForce', 4.9742923005211175), ('CustomBondForce', 34.42070082081463)]


INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=7.062600000000001, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=7.062600000000001), unit=nanometer)]
INFO:relati

In [4]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/5/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [5]:
import mdtraj as md

In [6]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [7]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# D39A

In [8]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor("../../input/1brs_barstar_renumbered.pdb",
                        '1', # First chain is the barstar one
                        '39',
                        'ALA',
                        ligand_input="../../input/1brs_barnase_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=OD1, atomic number=8), Atom(name=OD2, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 1
INFO:geometry:Atom index proposal order is [637]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old s

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 1 new atoms
INFO:geometry:	reduced angle potential = 0.6979325499735474.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 732 bond forces in the no-nonbonded final system
INFO:geometry:	there are 2632 angle forces in the no-nonbonded final system
INFO:geometry:	there are 4962 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 5670.107693081364


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 45.87756674237476
INFO:geometry:final reduced energy 5715.9852582222775
INFO:geometry:sum of energies: 5715.985259823739
INFO:geometry:magnitude of difference in the energies: 1.6014615127346588e-06
INFO:geometry:Final logp_proposal: 24.996447422263238


added energy components: [('CustomBondForce', 0.49949317692351586), ('CustomAngleForce', 4.6610804742915075), ('CustomTorsionForce', 13.826091963623828), ('CustomBondForce', 26.890901127535905)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [13]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/6/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [14]:
import mdtraj as md

In [15]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [16]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# H102A (with HIS in pdb, but the topology contains HIE), removed HIE from positive aminos in topology_proposal.py

In [7]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barnase_renumbered.pdb",
                        '1', # First chain is the barstar one
                        '102',
                        'ALA',
                        old_residue='HIE',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


Changed resid 102 to HIE


INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD2, atomic number=6), Atom(name=ND1, atomic number=7), Atom(name=CE1, atomic number=6), Atom(name=NE2, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HE1, atomic number=1), Atom(name=HE2, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]


current residue pdb:  /home/zhangi/miniconda3/envs/perses-nonstandard-aa/lib/python3.9/site-packages/perses-0.9.0+61.g4f830aa.dirty-py3.9.egg/perses/data/amino_acid_templates/HIE.pdb


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 1
INFO:geometry:Atom index proposal order is [1542]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 883 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 3089 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 5823 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching dist

...

INFO:geometry:total reduced potential before atom placement: 5643.035207714481


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 47.246860992463844
INFO:geometry:final reduced energy 5688.661147080358
INFO:geometry:sum of energies: 5690.282068706944
INFO:geometry:magnitude of difference in the energies: 1.6209216265863873
INFO:geometry:Final logp_proposal: 50.41541888746117


added energy components: [('CustomBondForce', 0.7062534695786508), ('CustomAngleForce', 21.90682773585043), ('CustomTorsionForce', 6.502502004638003), ('CustomBondForce', 18.13127778239676)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [8]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/14/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [9]:
import mdtraj as md

In [10]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [11]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# LYN27A (with LYS in PDB)

In [7]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barnase_renumbered_lyn27.pdb",
                        '1', # First chain is the barstar one
                        '27',
                        'ALA',
                        old_residue='LYN',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


Changed resid 27 to LYN


INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=CE, atomic number=6), Atom(name=NZ, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HD3, atomic number=1), Atom(name=HE2, atomic number=1), Atom(name=HE3, atomic number=1), Atom(name=HZ1, atomic number=1), Atom(name=HZ2, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]


current residue pdb filename:  /home/zhangi/miniconda3/envs/perses-nonstandard-aa/lib/python3.9/site-packages/perses-0.9.0+59.gecbc2e8.dirty-py3.9.egg/perses/data/amino_acid_templates/LYN.pdb


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 1
INFO:geometry:Atom index proposal order is [397]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 885 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 3078 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 5813 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching dista

...

INFO:geometry:total reduced potential before atom placement: 5680.188157741516


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -15.749000420903753
INFO:geometry:final reduced energy 5664.439155342768
INFO:geometry:sum of energies: 5664.439157320612
INFO:geometry:magnitude of difference in the energies: 1.9778446631590896e-06
INFO:geometry:Final logp_proposal: -5557.813158981127
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...


added energy components: [('CustomBondForce', 0.41235653061941924), ('CustomAngleForce', 3.519470373761121), ('CustomTorsionForce', 4.640212986437473), ('CustomBondForce', -24.321040311721767)]


INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=7.062600000000001, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=7.062600000000001), unit=nanometer)]
INFO:relati

In [8]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/8/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [9]:
import mdtraj as md

In [10]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [11]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# ASH39A  (with ASP in PDB)

In [12]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barstar_renumbered_ash39.pdb",
                        '1', # First chain is the barstar one
                        '39',
                        'ALA',
                        old_residue='ASH',
                        ligand_input="../../input/1brs_barnase_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


Changed resid 39 to ASH


INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=OD1, atomic number=8), Atom(name=OD2, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD2, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]


current residue pdb filename:  /home/zhangi/miniconda3/envs/perses-nonstandard-aa/lib/python3.9/site-packages/perses-0.9.0+59.gecbc2e8.dirty-py3.9.egg/perses/data/amino_acid_templates/ASH.pdb


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 1
INFO:geometry:Atom index proposal order is [637]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 732 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 2632 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 4962 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching dista

...

INFO:geometry:total reduced potential before atom placement: 5670.107693081364


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 8.81741233106282
INFO:geometry:final reduced energy 5678.92510194569
INFO:geometry:sum of energies: 5678.925105412427
INFO:geometry:magnitude of difference in the energies: 3.466737377166851e-06
INFO:geometry:Final logp_proposal: -6140.5057301147135
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...


added energy components: [('CustomBondForce', 10.788412589027248), ('CustomAngleForce', 1.9836023160449177), ('CustomTorsionForce', 33.10940081690952), ('CustomBondForce', -37.06400339091888)]


INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=7.062600000000001, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=7.062600000000001), unit=nanometer)]
INFO:relati

In [13]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/9/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [14]:
import mdtraj as md

In [15]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [16]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# R83Q

In [11]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barnase_renumbered.pdb",
                        '1', # First chain is the barstar one
                        '83',
                        'GLN',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE, atomic number=7), Atom(name=CZ, atomic number=6), Atom(name=NH1, atomic number=7), Atom(name=NH2, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HD3, atomic number=1), Atom(name=HE, atomic number=1), Atom(name=HH11, atomic number=1), Atom(name=HH12, atomic number=1), Atom(name=HH21, atomic number=1), Atom(name=HH22, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE2, atomic number=7), Atom(name=OE1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(n

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 5 new atoms
INFO:geometry:	reduced angle potential = 0.15041380929776985.
INFO:geometry:	reduced angle potential = 0.5197760956848299.
INFO:geometry:	reduced angle potential = 0.778474173079499.
INFO:geometry:	reduced angle potential = 0.14787206767908528.
INFO:geometry:	reduced angle potential = 0.36823210329413275.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 887 bond forces in the no-nonbonded final system
INFO:geometry:	there are 3090 angle forces in the no-nonbonded final system
INFO:geometry:	there are 5851 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined w

...

INFO:geometry:total reduced potential before atom placement: 5679.967596287112


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -483.62722285328334
INFO:geometry:final reduced energy 5196.340362341994
INFO:geometry:sum of energies: 5196.340373433829
INFO:geometry:magnitude of difference in the energies: 1.1091835517618165e-05
INFO:geometry:Final logp_proposal: 87.6931659281368


added energy components: [('CustomBondForce', 0.3697002478697511), ('CustomAngleForce', 7.494390739502455), ('CustomTorsionForce', 1.2626705861838576), ('CustomBondForce', -492.75398442683934)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [12]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/10/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [13]:
import mdtraj as md

In [14]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [15]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# R87A

In [16]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barnase_renumbered.pdb",
                        '1', # First chain is the barstar one
                        '87',
                        'ALA',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE, atomic number=7), Atom(name=CZ, atomic number=6), Atom(name=NH1, atomic number=7), Atom(name=NH2, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HD3, atomic number=1), Atom(name=HE, atomic number=1), Atom(name=HH11, atomic number=1), Atom(name=HH12, atomic number=1), Atom(name=HH21, atomic number=1), Atom(name=HH22, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propo

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 1 new atoms
INFO:geometry:	reduced angle potential = 0.5814477051984991.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 883 bond forces in the no-nonbonded final system
INFO:geometry:	there are 3078 angle forces in the no-nonbonded final system
INFO:geometry:	there are 5817 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 5679.2518762609825


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -480.8246242368886
INFO:geometry:final reduced energy 5198.427240882483
INFO:geometry:sum of energies: 5198.427252024094
INFO:geometry:magnitude of difference in the energies: 1.1141611196308077e-05
INFO:geometry:Final logp_proposal: 103.0276460117449


added energy components: [('CustomBondForce', 1.2299652445745632), ('CustomAngleForce', 6.061972930318527), ('CustomTorsionForce', 2.550543568365752), ('CustomBondForce', -490.66710598014754)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [17]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/11/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [18]:
import mdtraj as md

In [19]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [20]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# HID102A

In [22]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barnase_renumbered_hid102.pdb",
                        '1', # First chain is the barstar one
                        '102',
                        'ALA',
                        old_residue='HID',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


Changed resid 102 to HID


INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD2, atomic number=6), Atom(name=ND1, atomic number=7), Atom(name=CE1, atomic number=6), Atom(name=NE2, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD1, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HE1, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]


current residue pdb:  /home/zhangi/miniconda3/envs/perses-nonstandard-aa/lib/python3.9/site-packages/perses-0.9.0+61.g4f830aa.dirty-py3.9.egg/perses/data/amino_acid_templates/HID.pdb


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 1
INFO:geometry:Atom index proposal order is [1544]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 883 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 3089 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 5823 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching dist

...

INFO:geometry:total reduced potential before atom placement: 5643.035207714481


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 42.438762135101925
INFO:geometry:final reduced energy 5672.924069731686
INFO:geometry:sum of energies: 5685.473969849582
INFO:geometry:magnitude of difference in the energies: 12.549900117896314
INFO:geometry:Final logp_proposal: -499.325516646781


added energy components: [('CustomBondForce', 0.5854653845011346), ('CustomAngleForce', 26.30231398056091), ('CustomTorsionForce', 11.947970087744933), ('CustomBondForce', 3.6030126822949575)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [23]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/17/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [24]:
import mdtraj as md

In [25]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [26]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# HIP102A

In [27]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barnase_renumbered_hip102.pdb",
                        '1', # First chain is the barstar one
                        '102',
                        'ALA',
                        old_residue='HIP',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...


Changed resid 102 to HIP


INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD2, atomic number=6), Atom(name=ND1, atomic number=7), Atom(name=CE1, atomic number=6), Atom(name=NE2, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD1, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HE1, atomic number=1), Atom(name=HE2, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]


current residue pdb:  /home/zhangi/miniconda3/envs/perses-nonstandard-aa/lib/python3.9/site-packages/perses-0.9.0+61.g4f830aa.dirty-py3.9.egg/perses/data/amino_acid_templates/HIP.pdb


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 1
INFO:geometry:Atom index proposal order is [1544]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 883 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 3089 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 5823 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching dist

...

INFO:geometry:total reduced potential before atom placement: 5643.035207714482


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 56.34504789364118
INFO:geometry:final reduced energy 5711.216497032545
INFO:geometry:sum of energies: 5699.380255608123
INFO:geometry:magnitude of difference in the energies: 11.836241424421956
INFO:geometry:Final logp_proposal: -132.9993536652883


added energy components: [('CustomBondForce', 0.49589695589154287), ('CustomAngleForce', 27.69243593993732), ('CustomTorsionForce', 7.549368157571694), ('CustomBondForce', 20.607346840240634)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [28]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/18/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [29]:
import mdtraj as md

In [30]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [31]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# R83A

In [3]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barnase_renumbered.pdb",
                        '1', # First chain is the barstar one
                        '83',
                        'ALA',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE, atomic number=7), Atom(name=CZ, atomic number=6), Atom(name=NH1, atomic number=7), Atom(name=NH2, atomic number=7), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HD3, atomic number=1), Atom(name=HE, atomic number=1), Atom(name=HH11, atomic number=1), Atom(name=HH12, atomic number=1), Atom(name=HH21, atomic number=1), Atom(name=HH22, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propo

conducting subsequent work with the following platform: CPU
Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:There are 1 new atoms
INFO:geometry:	reduced angle potential = 0.6374922623883873.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 883 bond forces in the no-nonbonded final system
INFO:geometry:	there are 3078 angle forces in the no-nonbonded final system
INFO:geometry:	there are 5817 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


conducting subsequent work with the following platform: CPU


INFO:geometry:total reduced potential before atom placement: 2999.3002182720143


...

INFO:geometry:total reduced potential before atom placement: 5679.195004536044


conducting subsequent work with the following platform: CPU


INFO:geometry:total reduced energy added from growth system: -480.4630886277833
INFO:geometry:final reduced energy 5198.731910063705
INFO:geometry:sum of energies: 5198.73191590826
INFO:geometry:magnitude of difference in the energies: 5.8445552326702455e-06
INFO:geometry:Final logp_proposal: 107.01770866964348


Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.48963990548660075), ('CustomAngleForce', 8.042880777464282), ('CustomTorsionForce', 1.3669153355657713), ('CustomBondForce', -490.36252464629996)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=7.062600000000001, y=0.0, z=0.0), unit=nanometer)

In [4]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/20/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [5]:
import mdtraj as md

In [6]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [7]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))

# A83Q

In [8]:
# Load rhtf or generate it if it doesn't exist
solvent_delivery = PointMutationExecutor2("../../input/1brs_barnase_renumbered_ala83_cleaned.pdb",
                        '1', # First chain is the barstar one
                        '83',
                        'GLN',
                        ligand_input="../../input/1brs_barstar_renumbered.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD, atomic number=6), Atom(name=NE2, atomic number=7), Atom(name=OE1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HG2, atomic number=1), Atom(name=HG3, atomic number=1), Atom(name=HE21, atomic number=1), Atom(name=HE22, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom in

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 2.3411568661929376.
INFO:geometry:	reduced angle potential = 0.4072726699499721.
INFO:geometry:	reduced angle potential = 0.010936245829576604.
INFO:geometry:	reduced angle potential = 1.5152694933231163.
INFO:geometry:	reduced angle potential = 1.0302054296702565.
INFO:geometry:	reduced angle potential = 0.5642546224189988.
INFO:geometry:	reduced angle potential = 0.06143321547026684.
INFO:geometry:	reduced angle potential = 1.3726153913468973.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 887 bond forces in the no-nonbonded final system
INFO:geometry:

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


INFO:geometry:total reduced potential before atom placement: 2999.526840866948


...

INFO:geometry:total reduced potential before atom placement: 5679.421627130979


conducting subsequent work with the following platform: CPU


INFO:geometry:total reduced energy added from growth system: 3.881997583794174
INFO:geometry:final reduced energy 5683.303624755308
INFO:geometry:sum of energies: 5683.303624714773


Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1622798670214/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.3923835562589618), ('CustomTorsionForce', 0.0017567203609482687), ('CustomBondForce', 3.487857307174264)]


INFO:geometry:magnitude of difference in the energies: 4.053524094160821e-08
INFO:geometry:Final logp_proposal: 8.259143963655188
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
IN

In [9]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/20/21/"
i = os.path.basename(os.path.dirname(outdir))

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(outdir, f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)

apo_rhtf_0 = solvent_delivery.get_apo_rhtf_0()
with open(os.path.join(outdir, f"{i}_apo_0.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_0, f)

complex_rhtf_0 = solvent_delivery.get_complex_rhtf_0()
with open(os.path.join(outdir, f"{i}_complex_0.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_0, f)

apo_rhtf_1 = solvent_delivery.get_apo_rhtf_1()
with open(os.path.join(outdir, f"{i}_apo_1.pickle"), "wb") as f:
    pickle.dump(apo_rhtf_1, f)

complex_rhtf_1 = solvent_delivery.get_complex_rhtf_1()
with open(os.path.join(outdir, f"{i}_complex_1.pickle"), "wb") as f:
    pickle.dump(complex_rhtf_1, f)

# Render atom map
atom_map_filename = f'{outdir}/atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

In [10]:
import mdtraj as md

In [11]:
traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join(outdir, "complex_old.pdb"))

In [12]:
traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join(outdir, "complex_new.pdb"))